# NED Compare model to Brain Region

In [2]:
platform = "jupyter_notebook" # @param ["colab", "jupyter_notebook"] {allow-input: true}

In [3]:
!pip install -U git+https://github.com/gifale95/NED.git

  Cloning https://github.com/gifale95/NED.git to /private/var/folders/sy/39dkvhds36d6q24cvfwkcymc0000gn/T/pip-req-build-jm8jnjv1
  Running command git clone --quiet https://github.com/gifale95/NED.git /private/var/folders/sy/39dkvhds36d6q24cvfwkcymc0000gn/T/pip-req-build-jm8jnjv1
  Resolved https://github.com/gifale95/NED.git to commit cc238e51fdd117031d1096f104e712d0a4a95f46
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached nibabel-5.3.2-py3-none-any.whl.metadata (9.1 kB)
  Using cached scikit_learn-1.6.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (31 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached nibabel-5.3.2-py3-none-any.whl (3.3 MB)
Using cached scikit_learn-1.6.1-cp39-cp39-macosx_10_9_x86_64.whl (12.1 MB)
Using cached threadpoolctl

## import libraries

In [4]:
import h5py
import matplotlib
from matplotlib import pyplot as plt
from ned.ned import NED
import nibabel as nib
import numpy as np
import pandas as pd
import os
from PIL import Image
import torchvision
import torch
from torchvision import transforms as trn
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch import nn
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr, pearsonr

In [5]:
if platform == 'jupyter_notebook':
    ned_dir = './neural_encoding_dataset'

In [6]:
images_dir = os.path.join(ned_dir, 'ned_tutorials', 'tutorial_images')
model_dir = './cvp_models'  # Path to your models

<font color='red'><b>NOTE:</b></font> **Use only one of the models at a model.** 
Provided Models to compare are:
* **Fabina Model:** 
* **Curriculum (Simple Resnet):**
* **Layerwise (Simple Resnet):** 

## Fabian  model

Define the model, extract the features and set the model paths

In [51]:
## Fabian model
def get_model(num_classes=200):
    model = resnet18(weights=None)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()  # Remove maxpool
    model.fc = nn.Sequential(
        nn.Dropout(0.5),  # Dropout to avoid overfitting
        nn.Linear(model.fc.in_features, num_classes)
    )
    return model

In [52]:
# Function to extract features from your models
def extract_features(model_path, images):
    model = get_model()
    # Load the model state dictionary, mapping to CPU if necessary
    model.load_state_dict(torch.load(model_path, map_location=torch.device('mps')))
    model.eval()  # Set to evaluation mode
    with torch.no_grad():
        features = model(images).numpy()  # Extract features
    return features

In [53]:
model_paths = [os.path.join(model_dir, model_name) for model_name in [
    "resnet18_tinyimagenet_acuity.pth",
    "resnet18_tinyimagenet_contrast.pth",
    "resnet18_tinyimagenet_both.pth",
    "resnet18_tinyimagenet_default.pth"
]]

## Curriculum (Simple Resnet)

Define the model, extract the features and set the model paths

In [17]:
# # Sohan model
def get_resnet18(num_classes=200):
    model = resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

In [18]:
# Function to extract features from your models
def extract_features(model_path, images):
    model = get_resnet18()
    # Load the model state dictionary, mapping to CPU if necessary
    model.load_state_dict(torch.load(model_path, map_location=torch.device('mps')))
    model.eval()  # Set to evaluation mode
    with torch.no_grad():
        features = model(images).numpy()  # Extract features
    return features

In [19]:
model_paths = [os.path.join(model_dir, model_name) for model_name in [
    "resnet18_visual_acuity_final.pth",
    "resnet18_color_perception_final.pth",
    "resnet18_curriculum_final.pth",
    "resnet18_no_curriculum_final.pth"
]]

## Layerwise (simple resnet)

Define the model, extract the features and set the model paths

In [48]:
# # Sohan model
def get_resnet18(num_classes=200):
    model = resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

In [49]:
# Function to extract features from your models
def extract_features(model_path, images):
    model = get_resnet18()
    # Load the model state dictionary, mapping to CPU if necessary
    model.load_state_dict(torch.load(model_path, map_location=torch.device('mps')))
    model.eval()  # Set to evaluation mode
    with torch.no_grad():
        features = model(images).numpy()  # Extract features
    return features

In [50]:
model_paths = [os.path.join(model_dir, model_name) for model_name in [
    "resnet18_layerwise_acuity_final.pth",
    "resnet18_layerwise_color_final.pth",
    "resnet18_layerwise_final.pth",
    "resnet18_no_curriculum_final.pth"
]]

## Load and preprocess images

In [39]:
# Function to load and preprocess images
def load_images(images_dir, num_images=100):
    images_list = os.listdir(images_dir)
    images_list.sort()
    images_list = images_list[:num_images]  # Select first 100 images

    images = []
    for img in images_list:
        img_dir = os.path.join(images_dir, img)
        img = Image.open(img_dir).convert('RGB')
        transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
        ])
        img = transform(img)
        images.append(img)
    images = torch.stack(images)  # Stack into a single tensor
    return images, images_list

In [40]:
images, images_list = load_images(images_dir)

Define the NED object, subject, roi to use

In [45]:
ned_object = NED(ned_dir)
subject = 1 # @param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}

### Generate in silico fMRI responses to images

Generating neural responses for images involves two steps. First, you need to choose the training dataset, encoding model type, subject and ROI, and load the corresponding fMRI encoding model using the `get_encoding_model` method.

We provide fMRI encoding models for the following (NSD) ROIs:
* **Early retinotopic visual regions:** V1, V2, V3, hV4.
* **Body-selective regions:** EBA, FBA-2.
* **Face-selective regions:** OFA, FFA-1, FFA-2.
* **Place-selective regions:** OPA, PPA, RSC.
* **Word-selective regions:** OWFA, VWFA-1, VWFA-2, mfs-words.
* **Anatomical streams:** early, midventral, midlateral, midparietal, ventral, lateral, parietal.

For more information on the NSD ROIs, please see the [NSD data manual][nsd_man].

If you select encoding models trained on NSD, note that the fMRI data used to train and evaluate these encoding models were _z_-scored at each scan session. As a consequence, their generated in silico fMRI responses also live in _z_-scored space.

<font color='red'><b>NOTE:</b></font> **The in silico fMRI generation will be faster using if GPU is available.**

[nsd_man]: https://cvnlab.slite.page/p/X_7BBMgghj/ROIs

In [47]:
# Define the ROIs to compare
rois = ["V1", "V2", "V3", "hV4", "EBA", "FBA-2", "OFA", "FFA-1", "FFA-2", "early","midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]  # Add more ROIs as needed

# @param ["V1", "V2", "V3", "hV4", "EBA", "FBA-2", "OFA", "FFA-1", "FFA-2", "OPA", "PPA", "RSC", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", "early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]

## Results 
The results of Spearman and pearson correlation between the models and the brain regions

In [ ]:
# Initialize DataFrames to store results for both correlation methods
model_names = ["acuity model", "contrast model", "curriculum model", "no_curriculum model"]
results_spearman = pd.DataFrame(index=rois, columns=model_names)
results_pearson = pd.DataFrame(index=rois, columns=model_names)

# Iterate through each ROI and calculate similarities
for roi in rois:
    # Load the brain data for the current ROI
    brain_data, _ = ned_object.load_insilico_neural_responses(
        modality='fmri',
        train_dataset='nsd',
        model='fwrf',
        imageset='nsd',
        subject=subject,
        roi=roi,
        return_metadata=True
    )
    
    # Calculate the brain RDM for the current ROI
    brain_rdm = squareform(pdist(brain_data[:100], metric='correlation'))
    model_features = [extract_features(model_path, images) for model_path in model_paths]
    model_rdms = [squareform(pdist(features, metric='correlation')) for features in model_features]
    
    # Compare each model to the brain RDM using different correlations
    for i, model_rdm in enumerate(model_rdms):
        # Flatten the RDMs for correlation calculations
        brain_rdm_flat = brain_rdm.flatten()
        model_rdm_flat = model_rdm.flatten()

        # Spearman correlation
        spearman_corr, _ = spearmanr(brain_rdm_flat, model_rdm_flat)
        results_spearman.loc[roi, model_names[i]] = spearman_corr
        

        # Pearson correlation
        pearson_corr, _ = pearsonr(brain_rdm_flat, model_rdm_flat)
        results_pearson.loc[roi, model_names[i]] = pearson_corr
        
# Save the results to CSV files
results_spearman.to_csv("spearman_correlations.csv")
results_pearson.to_csv("pearson_correlations.csv")

# Display the results as tables
print("Spearman Correlations:")
print(results_spearman)

print("\nPearson Correlations:")
print(results_pearson)

# Provide download links (if running in a Jupyter-like environment)
import shutil

# Copy files to a directory accessible for download
output_dir = "./data/"
os.makedirs(output_dir, exist_ok=True)
shutil.move("spearman_correlations.csv", "./data/spearman_correlations.csv")
shutil.move("pearson_correlations.csv", "./data/pearson_correlations.csv")
print("Download Spearman Correlations: ./data/spearman_correlations.csv")
print("Download Pearson Correlations: ./data/pearson_correlations.csv")


Spearman Correlations:
            acuity model contrast model curriculum model no_curriculum model
V1              0.030257        0.01025         0.027624           -0.000826
V2              0.042575       0.017419         0.035278            0.007411
V3               0.02523       0.018588         0.027911            0.017775
hV4             0.025388       0.020933         0.018436            0.004119
EBA             0.018718       0.028583         0.029167            0.023601
FBA-2           0.026583       0.040613         0.037381             0.05279
OFA             0.023218       0.054792         0.038804            0.060731
FFA-1           0.021205       0.014854         0.023721             0.02626
FFA-2           0.010004       0.020683         0.016646            0.026218
early           0.035069        0.04085         0.039115            0.040625
midventral      0.022708       0.021428         0.015438            0.011751
midlateral      0.022705       0.021953         0.026